#### **This notebook is designed to quickly evaluate a given approach with Replicate Out Cross-Validation**

Replicate Out Cross-Validation, uses surface as a testing set, training on the rest.

This gives a better indication of the model's ability to generalise, as it stops leakage between samples from the same fold.

This [paper](https://pubs.rsc.org/en/content/articlelanding/2017/ay/c7ay01363a) compares KFold and Replicate Out Cross-Validation in the domain of Raman Spectroscopy.

Import Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold
import seaborn as sns
import sys
sys.path.append('..')
from Cleaning_and_Evaluation import *
import matplotlib.pyplot as plt

Read the spectral data

In [2]:
# df = pd.read_csv("../../data/exosomes.raw_spectrum_400-1800.csv")
df = pd.read_csv("../../data/400-1800_with_raw_scaled_surface_pagerank.csv")

In [3]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status,PageRank
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal,0.610024
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal,0.610024
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal,0.610024
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal,0.610024
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal,0.610024
...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia,1.201164
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia,1.201164
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia,1.201164
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia,1.201164


---

#### **Select the chosen cleaning parameters then run the functions**

Choose the Parameters

In [4]:
# # Best Full Spectrum Parameters

# Optuna Outlier Removed Parameters
pagerank_cutoff = 0.902461
despike_ma = 57
despike_threshold = 3.75
lam = 10 ** 11
p = 0.074
window_size = 51
poly_order = 3

# pagerank_cutoff = 0.902461
# despike_ma = 57
# despike_threshold = 3.75
# lam = 10 ** 8
# p = 0.056
# lam_1 = 10**-4
# window_size = 19
# poly_order = 2

#### **Here is an example on 1 Spectra**

In [10]:
df['Absorbance'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: despike_group(x, ma=despike_ma, threshold=despike_threshold))

In [11]:
df['Baseline'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam=lam, p=p))
#df['Baseline'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: iasls_baseline_correction(x, lam=lam, p=p, lam_1=lam_1))

In [12]:
df['Absorbance'] = df['Absorbance'] - df['Baseline']

In [13]:
# df['Absorbance'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: savgol_filter(x, window_size, poly_order, deriv=0))

In [14]:
# df['Absorbance'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: normalise(x))
df['Absorbance'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: vector_normalise(x))
# df['Absorbance'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: svn_normalise(x))

In [15]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status,PageRank,Baseline
0,201210-1-00,293,400.22778,-0.001247,201210-1,Normal,0.610024,1769.569028
1,201210-1-00,294,400.91116,0.001740,201210-1,Normal,0.610024,1769.330009
2,201210-1-00,295,401.59454,-0.000019,201210-1,Normal,0.610024,1769.090991
3,201210-1-00,296,402.27789,-0.003967,201210-1,Normal,0.610024,1768.851972
4,201210-1-00,297,402.96127,-0.003110,201210-1,Normal,0.610024,1768.612953
...,...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,-0.016926,210526-3,Hyperglycemia,1.201164,1753.639314
6239201,210526-3-09,2338,1797.72200,-0.014938,210526-3,Hyperglycemia,1.201164,1753.341720
6239202,210526-3-09,2339,1798.40550,-0.014875,210526-3,Hyperglycemia,1.201164,1753.044127
6239203,210526-3-09,2340,1799.08890,-0.013774,210526-3,Hyperglycemia,1.201164,1752.746533


---

#### **Evaluating the approach**

In [22]:
from sklearn.model_selection import cross_validate, GroupKFold
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report

In [23]:
def evaluate_model(df, model):

    # Set the Surfaces as groups
    groups = df['SurID']
    X = df.drop(['Status', 'SurID'], axis=1)
    y = df['Status']
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    # Cross Validate
    scores = cross_validate(model, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], n_jobs=-1)

    # Displaying the results
    print(f"{model.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores['test_accuracy']):.4f} +/- {np.std(scores['test_accuracy']):.4f}")
    print(f"{model.__class__.__name__} Cross-Validation Precision: {np.mean(scores['test_precision_macro']):.4f} +/- {np.std(scores['test_precision_macro']):.4f}")
    print(f"{model.__class__.__name__} Cross-Validation Recall: {np.mean(scores['test_recall_macro']):.4f} +/- {np.std(scores['test_recall_macro']):.4f}")
    print(f"{model.__class__.__name__} Cross-Validation F1-Score: {np.mean(scores['test_f1_macro']):.4f} +/- {np.std(scores['test_f1_macro']):.4f}")

In [24]:
rf = RandomForestClassifier(random_state=1234, criterion='entropy', max_depth=23, n_estimators=78)
wavelength_df = prepare_wavelength_df(df, 'Absorbance')

evaluate_model(wavelength_df, rf)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForestClassifier Cross-Validation Accuracy: 0.5934 +/- 0.1085
RandomForestClassifier Cross-Validation Precision: 0.5971 +/- 0.1154
RandomForestClassifier Cross-Validation Recall: 0.6086 +/- 0.1485
RandomForestClassifier Cross-Validation F1-Score: 0.5674 +/- 0.1325


Try After PageRank Outlier Removal

In [25]:
df = df[df['PageRank'] > pagerank_cutoff]
wavelength_df = prepare_wavelength_df(df, 'Absorbance')
evaluate_model(wavelength_df, rf)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForestClassifier Cross-Validation Accuracy: 0.6966 +/- 0.1481
RandomForestClassifier Cross-Validation Precision: 0.6684 +/- 0.1079
RandomForestClassifier Cross-Validation Recall: 0.6575 +/- 0.1309
RandomForestClassifier Cross-Validation F1-Score: 0.6280 +/- 0.1514


In [26]:
def evaluate_random_forest_by_class(df):

    groups = df['SurID']
    X = df.drop(['Status', 'SurID'], axis=1)
    y = df['Status']
    
    # Get unique class names from the 'y' dataset
    class_names = y.unique()

    rf = RandomForestClassifier(random_state=1234, criterion='entropy', max_depth=23, n_estimators=78)
    cv = GroupKFold(n_splits=10)

    accuracies = []
    precisions = [[] for _ in class_names]
    recalls = [[] for _ in class_names]
    f1_scores = [[] for _ in class_names]

    for train_idx, test_idx in cv.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        accuracies.append(accuracy_score(y_test, y_pred))
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, zero_division=0, labels=class_names)
        for i, (p, r, f) in enumerate(zip(precision, recall, f1)):
            precisions[i].append(p)
            recalls[i].append(r)
            f1_scores[i].append(f)

    print(f"{rf.__class__.__name__} Cross-Validation Accuracy: {np.mean(accuracies):.4f} +/- {np.std(accuracies):.4f}\n")
    for i, class_name in enumerate(class_names):
        print(f"{class_name} Precision: {np.mean(precisions[i]):.4f} +/- {np.std(precisions[i]):.4f}")
        print(f"{class_name} Recall: {np.mean(recalls[i]):.4f} +/- {np.std(recalls[i]):.4f}")
        print(f"{class_name} F1-Score: {np.mean(f1_scores[i]):.4f} +/- {np.std(f1_scores[i]):.4f}\n")

In [27]:
evaluate_random_forest_by_class(wavelength_df)

In [ ]:
df.groupby('Status')['SpecID'].nunique()

Status
Hyperglycemia    502
Hypoglycemia     653
Normal           628
Name: SpecID, dtype: int64

---

#### **Finally Try to Predict the Surface by its predicted Spectra**

In [ ]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status,PageRank,Baseline
4098,201210-1-02,293,400.22778,0.017863,201210-1,Normal,0.961147,2041.552352
4099,201210-1-02,294,400.91116,0.013682,201210-1,Normal,0.961147,2041.568641
4100,201210-1-02,295,401.59454,0.013220,201210-1,Normal,0.961147,2041.584930
4101,201210-1-02,296,402.27789,0.014904,201210-1,Normal,0.961147,2041.601219
4102,201210-1-02,297,402.96127,0.015908,201210-1,Normal,0.961147,2041.617508
...,...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,-0.016926,210526-3,Hyperglycemia,1.201164,1753.639314
6239201,210526-3-09,2338,1797.72200,-0.014938,210526-3,Hyperglycemia,1.201164,1753.341720
6239202,210526-3-09,2339,1798.40550,-0.014875,210526-3,Hyperglycemia,1.201164,1753.044127
6239203,210526-3-09,2340,1799.08890,-0.013774,210526-3,Hyperglycemia,1.201164,1752.746533


In [ ]:
def evaluate_and_predict_surface_status(df):
   # Set the Surfaces as groups
    groups = df['SurID']
    X = df.drop(['Status', 'SurID'], axis=1)
    y = df['Status']

    # Creating the RandomForestClassifier
    rf = RandomForestClassifier(random_state=1234, criterion='entropy', max_depth=23, n_estimators=78)
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)
    
    # DataFrame to store the actual and predicted status for each spectrum
    predictions_df = pd.DataFrame(columns=['SurID', 'ActualStatus', 'PredictedStatus'])

    # Iterating over each fold
    for train_index, test_index in cv.split(X, y, groups=groups):
        # Splitting the dataset into the current train and test set
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        groups_test = groups.iloc[test_index]
        
        # Fitting the model on the training set
        rf.fit(X_train, y_train)

        # Predicting the status for the test set
        y_pred = rf.predict(X_test)

        # Appending the SurID, actual, and predicted status to the predictions DataFrame
        fold_predictions = pd.DataFrame({
            'SurID': groups_test.values,
            'ActualStatus': y_test.values,
            'PredictedStatus': y_pred
        })
        predictions_df = pd.concat([predictions_df, fold_predictions], ignore_index=True)

    # Return the most common prediction of each spectra within the surface
    return predictions_df.groupby('SurID').agg(lambda x: x.mode()[0]).reset_index()

In [ ]:
surface_predictions = evaluate_and_predict_surface_status(wavelength_df)
surface_predictions

,SurID,ActualStatus,PredictedStatus
0,201210-1,Normal,Normal
1,201210-2,Normal,Hypoglycemia
2,210114-1,Normal,Normal
3,210114-2,Normal,Normal
4,210120-1,Hyperglycemia,Normal
...,...,...,...
58,210519-3,Hyperglycemia,Hyperglycemia
59,210524-1,Hypoglycemia,Hyperglycemia
60,210526-1,Hyperglycemia,Hyperglycemia
61,210526-2,Hyperglycemia,Normal


In [ ]:
print(classification_report(surface_predictions['ActualStatus'], surface_predictions['PredictedStatus']))

               precision    recall  f1-score   support

Hyperglycemia       0.70      0.74      0.72        19
 Hypoglycemia       0.72      0.82      0.77        22
       Normal       0.78      0.64      0.70        22

     accuracy                           0.73        63
    macro avg       0.73      0.73      0.73        63
 weighted avg       0.73      0.73      0.73        63

